# Creating a Custom Container

This notebook builds a custom container that we can deploy to a SageMaker endpoint. It uses `docker` commands directly and therefore doesn't run from a SageMaker Studio notebook.

In [1]:
import uuid
import sagemaker
import boto3

from datetime import datetime

region = boto3.Session().region_name

In [2]:
VERSION = "1"

ENDPOINTS_FOLDER = "version" + VERSION
REPOSITORY_NAME = "penguins"

CONTAINER_VERSION = f"{datetime.now().year}{str(datetime.now().month).zfill(2)}{str(datetime.now().day).zfill(2)}{str(uuid.uuid4())[-4:]}" 
print(f"Container version: {CONTAINER_VERSION}")

Container version: 20230412791e


In [3]:
!aws ecr create-repository --repository-name $REPOSITORY_NAME


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'penguins' already exists in the registry with id '325223348818'


In [4]:
repository = !aws ecr describe-repositories \
    --repository-names $REPOSITORY_NAME \
    --query "repositories[0].repositoryUri"

repository_uri = repository[0][1:-1]
repository = repository_uri[0:repository_uri.index("/")]

CUSTOM_CONTAINER_IMAGE_URI = f"{repository}/{REPOSITORY_NAME}:CONTAINER_VERSION"
print(f"CUSTOM_CONTAINER_IMAGE_URI = {CUSTOM_CONTAINER_IMAGE_URI}")

ECR Repository: 325223348818.dkr.ecr.us-east-1.amazonaws.com/penguins


In [5]:
!aws ecr get-login-password --region $region | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

!docker build -t $repository_uri:$CONTAINER_VERSION .
!aws ecr get-login-password | docker login --username AWS --password-stdin $repository
!docker push $repository_uri:$CONTAINER_VERSION

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  86.53kB
Step 1/10 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.4-cpu AS sagemaker
2.4-cpu: Pulling from tensorflow-inference

53061382: Pulling fs layer 
08ed77a3: Pulling fs layer 
ce9f2a54: Pulling fs layer 
e17cc9e8: Pulling fs layer 
b8fc7df5: Pulling fs layer 
ee81db4a: Pulling fs layer 
a4286210: Pulling fs layer 
fd9e2ccc: Pulling fs layer 
619b3af0: Pulling fs layer 
d0a17937: Pulling fs layer 
7dc240a4: Pulling fs layer 
641e5328: Pulling fs layer 
9c4f05f6: Pulling fs layer 
d8206bc8: Pulling fs layer 
0dfee059: Pulling fs layer 
1cf7edab: Pull complete .39kB/38.39kBB[16ADigest: sha256:9655faca3ac7c26a9583ad05a324ca7a58da4af1c14f3d2daae0d217688356a9
Status: Downloaded n